### Twilight Struggle

In [1]:
from twilight_ui import *

### Game Instance

In [2]:
u = UI()
u.run()

Initalising game..
> new
Starting new game.
----- USSR Player: -----
Place starting influence.
Remaining influence: 1
Available options:
   18 Czechoslovakia
    7 Finland
   14 Austria
   19 Hungary
   17 Poland
   16 East_Germany
   20 Yugoslavia
   21 Romania
   22 Bulgaria
> m 14
Selected: Austria
----- USSR Player: -----
Place starting influence.
You have selected Austria
Remaining influence: 0
Commit your actions? (Yes/No)
> m y
----- US Player: -----
Place starting influence.
Remaining influence: 1
Available options:
   12 Italy
    7 Finland
   11 Spain_Portugal
    3 Canada
   10 France
   14 Austria
   23 Turkey
   15 West_Germany
    8 Denmark
    9 Benelux
    4 UK
    5 Norway
   13 Greece
    6 Sweden
> m 7
Selected: Finland
----- US Player: -----
Place starting influence.
You have selected Finland
Remaining influence: 0
Commit your actions? (Yes/No)
> m y
----- US Player: -----
Place additional starting influence.
Remaining influence: 2
Available options:
    3 Canada
  

In [6]:
u.game.map['Taiwan'].info.battleground

True